In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [3]:
# tool create 

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two numbers."""
    return a * b

In [5]:
print(multiply.invoke({'a': 3, 'b': 3}))

9


In [6]:
multiply.name

'multiply'

In [7]:
multiply.description

'Multiplies two numbers.'

In [8]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

## Tool Binding

In [9]:
llm = ChatOpenAI()

In [ ]:
llm.invoke("hi")

In [ ]:
llm_with_tools = llm.bind_tools([multiply])

In [ ]:
llm_with_tools.invoke("Hi")

In [ ]:
query = HumanMessage('can you multiply 3 with 1000')

In [ ]:
messages = [query]

In [ ]:
result = llm_with_tools.invoke(messages)

In [ ]:
messages.apend(result)

In [ ]:
tool_result = multiply.invoke(result.tool_calls[0])

In [ ]:
messages.append(tool_result)

In [ ]:
llm_with_tools.invoke(messages).content

## Currency Conversion Tool

In [11]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

In [16]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """Get the conversion factor between two currencies."""
    url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'
    response = requests.get(url)
    data = response.json()
    return data

@tool
def convert(base_currency: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """Convert a base currency to a target currency using the conversion rate."""
    return base_currency * conversion_rate

In [15]:
get_conversion_factor.invoke({
    'base_currency': 'USD',
    'target_currency': 'PKR'
})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1747872002,
 'time_last_update_utc': 'Thu, 22 May 2025 00:00:02 +0000',
 'time_next_update_unix': 1747958402,
 'time_next_update_utc': 'Fri, 23 May 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'PKR',
 'conversion_rate': 281.9998}

In [18]:
convert.invoke({
    'base_currency': 100,
    'conversion_rate': 281.9998
})

28199.98

In [19]:
# tool binding
llm = ChatOpenAI()

In [20]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [ ]:
messages = [HumanMessage('What is the conversion factor between PKR and USD, and based on that can you convert 10 pkr to usd')]

In [22]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [ ]:
ai_messages = llm_with_tools.invoke(messages)

In [ ]:
messages.append(ai_messages)

In [ ]:
ai_messages.tool_calls

In [ ]:
import json

for tool_call in ai_messages.tool_calls:
    # execute the 1st tool and get the value of conversion rate
    if tool_call.tool.name == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        # fetch this conversion rate
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        # append this tool message to messages list
        messages.append(tool_message1)
    # execute the 2nd tool and get the value of converted currency
    if tool_call.tool.name == 'convert':
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message2 = convert.invoke(tool_call)
        # append this tool message to messages list
        messages.append(tool_message2)

In [ ]:
messages

In [ ]:
llm_with_tools.invoke(messages).content

In [ ]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

In [ ]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})